# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record


# Project Approach

## Problem Statement

For this project a model will be built that predicts whether a customer will view and complete an offer based on the customer information and the offer characteristics. This prediction can then be used to identify the best offer to a customer based on how likely it is that the customer will be influenced by the offer. 
Any offer can fall into one of four categories on the viewed/completion matrix: 

|----------|Completed|Not Completed|
|----------|---------|-------------|
|Viewed|1|2|
|Not Viewed|4|3|

The value of each category can be ranked from most desirable (1) to least desirable (4). Based on this a simple recommendation function can be built that uses the insights gained from the models predictions. 


## Project Design

In a first step the datasets will be cleaned. This means dropping any customer profiles that are incomplete and the transactions recorded from these incomplete customer profiles. Next the transaction volume per customer per week will be added as additional information to the customer profiles. Lastly, a dataset that records all the offers sent to customers and their outcomes (i.e. whether they were viewed and whether they were completed) will be constructed. For every offer sent the personal information of the customer will be added to the offer dataset to construct the input dataset explained above. 
After the dataset has been split into test and training set, the Benchmark model will be trained to establish the Benchmark. 
Next several different Machine Learning models will be trained and tested to see which model performs best based on the evaluation metrics. 
Lastly, once the best model has been picked, a sample recommendation function will be introduced. This recommendation sample will take the customer ID as input and then predict the outcomes for the available offers for this customers. Once the outcomes are known a predetermined ranking will identify which offer is most suitable for the customer and this offer will be returned.
To achieve this the project has been split into the following sections:
0. Data Exploration
1. Data Pre-Processing
2. Data Exploration 2.0
3. Benchmark Model
4. Machine Learning Models
5. Sample Recommendation Function


In [93]:
import pandas as pd
import numpy as np
import math
import json
%matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

# 0. Data Exploration
Before jumping into the project, it makes sense to take a quick look at the available datasets to get a better feel for the data. In the following the form of the datasets and whether or not the datasets are complete. As these datasets will be changed significantly before using them for the model, a more detailed data exploration section will follow after the Data Pre-Processing.


In [131]:
portfolio.head(10)

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [17]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [235]:
# Shape of datasets
print('Shape of Profile Dataset: {}'.format(profile.shape))
print('Shape of Transcript Dataset: {}'.format(transcript.shape))

Shape of Profile Dataset: (17000, 5)
Shape of Transcript Dataset: (306534, 4)


In [237]:
#Missing Values
print('Missing values in Profile Dataset: {}'.format(profile.isnull().sum().sum()))
print('Missing values in Transcript Dataset: {}'.format(transcript.isnull().sum().sum()))

Missing values in Profile Dataset: 4350
Missing values in Transcript Dataset: 0


# 1. Data Pre-Processing
The goal of the data pre-processing is to obtain a Dataset that can be used to train a supervised learning model. The resulting dataset should consist of a label (i.e., which category in the viewed/completed matrix the observation falls into) and a number of input variables including the available information about a customer, the offer type and the average weekly transaction volumes of a customer. 

To achieve this goal a number of steps have to be taken: 
1. Data Cleaning
2. Construction of Source Datasets from the available Data
3. Conversion to machine readable datatypes
4. Feature Scaling

## 1.1 Data Cleaning

The first step in Data Pre-Processing is the data cleaning step. For the available datasets this is limited to dropping any incomplete cells. Only the profile dataframe has observations with incomplete data. However, as the transcript dataframe also records transactions of customers that have an incomplete profile, the transcript dataframe, all entries in the transcript dataframe that relate to an incomplete customer profile have to be dropped.



In [110]:
def data_cleaning(profile_df, transcript_df):
    ''' Function for dropping all incomplete entries from the profile dataframe and also deleting the 
    corresponding entries of the transcript dataframe
    Parameters: 
                profile_df - pandas.DataFrame holding customer data
                transcript_df - pandas.DataFrame holding transaction data
    Output: 
                clean_profile_df - profile_df excluding all rows that have NA vlaues
                clean_transcript_df - transcript_df excluding all rows refering to a dropped row from profile_df
    '''
    #store current number of entries in transcript_df and profile_df for later use
    num_rows_t = transcript_df.shape[0]
    num_rows_p = profile_df.shape[0]
    
    # get a dataframe with all observations in profile_df that have missing values
    incomplete = profile_df[profile_df.isnull().any(axis = 1)]
    # for every entry in the incomplete dataframe, drop all correspronding entries from the transcript_df 
    for index, row in incomplete.iterrows():
        #store person_id as string
        ident = str(row['id'])
        # create dataframe where all entries that have the incomplete id are set to np.Nan
        transcript_df.where(transcript_df['person'] != ident, inplace = True)
        # drop the entries that were just set to np.Nan
        transcript_df.dropna(axis = 0, inplace = True)
        
    # drop all incomplete rows from profile_df now that the transcript is cleaned 
    profile_df.dropna(axis = 0, inplace = True)
    
    # show how many lines were dropped in total
    print('{} Entries were dropped from transcript'.format(num_rows_t - transcript_df.shape[0])) 
    print('{} Entries were dropped from profile'.format(num_rows_p - profile_df.shape[0]))
    
    #return cleaned version of profile_df and transcript_df
    return profile_df, transcript_df
    

In [106]:
#Test the cleaning function
profile_test = profile.head(100)
transcript_test = transcript.copy()

clean_profile, clean_transcript = data_cleaning(profile_test, transcript_test)

333 Entries were dropped from transcript
21 Entries were dropped from profile
(306201, 4)


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [107]:
profile_test = None
transcript_test = None

In [108]:
# Excecute the cleaning function and store the cleaned datasets
clean_profile, clean_transcript = data_cleaning(profile.copy(), transcript.copy())

33772 Entries were dropped from transcript
2175 Entries were dropped from profile
(272762, 4)


In [111]:
clean_profile.shape

(14825, 5)

## 1.2 Construction of Input Datasets from the available Data

Now that the datasets only contain complete datapoints, the next step is to turn the datasets into a single dataframe that can be used to train a model. Once again the input dataset should consist of information about the customer and the offer type. The label is the category of the viewed/completed matrix the particular offer falls into. The customer information is made up of the information in the clean_profile dataframe as well as information about the transaction volume of the customer. The offer data comes from the portfolio dataframe. Most importantly, each offer that was received has to be filtered out of the transcript dataframe and put into a new dataframe where it can be sorted to show which offer was viewed and completed. 

The first step will be to sort through the clean_transcript dataframe and create a new offer_df dataframe that holds every offer that was sent to customers and whether this offer was viewed and completed. The dataframe thus requires four columns: a customer id, the offer id, a binary variable 'viewed' with 0 for not viewed and 1 for viewed and a binary variable 'completed' with 0 for not completed and 1 for completed. 
At a later stage this dataframe can then be extended with the additional information. An offer only counts as viewed if it was viewed before it was completed.

To obtain this dataset two functions will be used. The first function __create_customer_offers__ will generate a dataframe with the columns described above for one single customer and hold the information for all the offers this customer received. The second function __create_offer_overview__ will then iterate over every customer and call the __create_customer_offers__ to built a dataframe that holds all the offers recorded in the clean_transcript dataframe.

In [114]:
test_data = clean_transcript.where(clean_transcript['person'] == '78afa995795e4d85b5d9ceeca43f5fef')
test_data.dropna(axis = 0, inplace = True)
print(test_data)


                                  person            event  \
0       78afa995795e4d85b5d9ceeca43f5fef   offer received   
15561   78afa995795e4d85b5d9ceeca43f5fef     offer viewed   
47582   78afa995795e4d85b5d9ceeca43f5fef      transaction   
47583   78afa995795e4d85b5d9ceeca43f5fef  offer completed   
49502   78afa995795e4d85b5d9ceeca43f5fef      transaction   
53176   78afa995795e4d85b5d9ceeca43f5fef   offer received   
85291   78afa995795e4d85b5d9ceeca43f5fef     offer viewed   
87134   78afa995795e4d85b5d9ceeca43f5fef      transaction   
92104   78afa995795e4d85b5d9ceeca43f5fef      transaction   
141566  78afa995795e4d85b5d9ceeca43f5fef      transaction   
150598  78afa995795e4d85b5d9ceeca43f5fef   offer received   
163375  78afa995795e4d85b5d9ceeca43f5fef     offer viewed   
201572  78afa995795e4d85b5d9ceeca43f5fef   offer received   
218393  78afa995795e4d85b5d9ceeca43f5fef      transaction   
218394  78afa995795e4d85b5d9ceeca43f5fef  offer completed   
218395  78afa995795e4d85

In [119]:
#split into four dataframes
received_df = test_data.where(test_data['event'] == 'offer received')
received_df.dropna(axis = 0, inplace = True)
received_df.head(10)

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0.0
53176,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},168.0
150598,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': 'ae264e3637204a6fb9bb56bc8210ddfd'},408.0
201572,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},504.0


In [120]:
viewed_df = test_data.where(test_data['event'] == 'offer viewed')
viewed_df.dropna(axis = 0, inplace = True)
viewed_df.head(10)

,person,event,value,time
15561,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},6.0
85291,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},216.0
163375,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,{'offer id': 'ae264e3637204a6fb9bb56bc8210ddfd'},408.0
262138,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},582.0


In [121]:
completed_df = test_data.where(test_data['event'] == 'offer completed')
completed_df.dropna(axis = 0, inplace = True)
completed_df.head(10)

,person,event,value,time
47583,78afa995795e4d85b5d9ceeca43f5fef,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,132.0
218394,78afa995795e4d85b5d9ceeca43f5fef,offer completed,{'offer_id': 'ae264e3637204a6fb9bb56bc8210ddfd...,510.0
218395,78afa995795e4d85b5d9ceeca43f5fef,offer completed,{'offer_id': 'f19421c1d4aa40978ebb69ca19b0e20d...,510.0


In [215]:
def create_customer_offers(received_df, viewed_df, completed_df, portfolio):
    ''' Function for creating a new dataframe that records all the offers that were sent to a single customer and
    record whether the offer was viewed and completed.
    Parameters:
                received_df - pandas.DataFrame holding the records for all received offers
                viewed_df - pandas.DataFrame holding the records for all viewed offers
                completed_df - pandas.DataFrame holding the records for all completed offers
                portfolio - pandas.DataFrame holding the details about the different offer types
                
    Output: 
                customer_offers - pandas.DataFrame that holds infromation on each offer made to customers
    '''
    
    column_names = ['person', 'offer_id', 'viewed', 'completed']
    customer_offers = pd.DataFrame(columns = column_names)
    # make an entry
    for index, row in received_df.iterrows():
        #create a dictionary with the entries
        entry = {}
        #set all variables that might not be changed to None
        time_com = time_view = None
        
        #get person id of received offer
        entry['person'] = row['person']
        
        #get offer id and time of received offer
        offer_ident = row['value']['offer id']
        time_rec = int(row['time'])
        #calculate valid duration time in hours
        dur = portfolio.loc[portfolio['id'] == offer_ident]['duration'].squeeze() * 24
        #update in dictionary
        entry['offer_id'] = offer_ident
        #fill completed column with 0 if not completed and 1 if completed
        completed = 0
        for index, row in completed_df.iterrows():
            #store the time of completion
            time_com = int(row['time'])
            # if the received offer id appears in the completed section and the time difference is not bigger
            # than the offer duration, set completed to 1
            if str(row['value']['offer_id']) == str(offer_ident) and time_com - time_rec < dur:
                completed = 1
                break
        # update completed entry
        entry['completed'] = completed
        
        #fill viewed column with 0 if not viewed and 1 if viewed
        viewed = 0
        for index, row in viewed_df.iterrows():
            #store the time viewed
            time_view = int(row['time'])
            # if the received offer id appears in the viewed section and the view time is before the completion time
            # set viewed to 1
            if str(row['value']['offer id']) == str(offer_ident): 
                #if there is no completion time set viewed to 1
                if time_com == None:
                    viewed = 1
                    break
                #if there is a completion time, check if completion time is after view time
                elif time_view < time_com:
                    viewed = 1
                    break
        # update viewed entry
        entry['viewed'] = viewed
        
        #add entry to the dataframe
        customer_offers = customer_offers.append(entry, ignore_index=True)
        
        
    #return dataframe
    return customer_offers
        


In [176]:
for index, row in received_df.iterrows():
    offer_id = row['value']['offer id']
    dur = portfolio.loc[portfolio['id'] == offer_id]['duration'].squeeze()
    print(offer_id)
    print(dur)

9b98b8c7a33c4b65b9aebfe6a799e6d9
7
5a8bc65990b245e5a138643cd4eb9837
3
ae264e3637204a6fb9bb56bc8210ddfd
7
f19421c1d4aa40978ebb69ca19b0e20d
5


In [217]:
new = create_df(received_df, viewed_df, completed_df, portfolio)
print(new)

                             person                          offer_id viewed  \
0  78afa995795e4d85b5d9ceeca43f5fef  9b98b8c7a33c4b65b9aebfe6a799e6d9      1   
1  78afa995795e4d85b5d9ceeca43f5fef  5a8bc65990b245e5a138643cd4eb9837      1   
2  78afa995795e4d85b5d9ceeca43f5fef  ae264e3637204a6fb9bb56bc8210ddfd      1   
3  78afa995795e4d85b5d9ceeca43f5fef  f19421c1d4aa40978ebb69ca19b0e20d      0   

  completed  
0         1  
1         0  
2         1  
3         1  


In [214]:
a = b = None

In [220]:
def create_offer_overview(profile_df, transcript_df, portfolio):
    ''' Function for creating a new dataframe that records all the offers that were sent and whether they were
    viewed and completed. The function iterates over every customer and then calls the create_customer_offers 
    function to get the relevant DataFrame entries for the specific customer. It then adds these entries to 
    the offer_df DataFrame.
    Parameters:
                profile_df - pandas.DataFrame holding customer data
                transcript_df - pandas.DataFrame holding transaction data
                portfolio - pandas.DataFrame holding the details about the different offer types
    Output: 
                offer_df - pandas.DataFrame that holds infromation on each offer made to customers
    '''
    #Create the output Dataframe object:offer_df
    column_names = ['person', 'offer_id', 'viewed', 'completed']
    offer_df = pd.DataFrame(columns = column_names)
    
    #Iterate over every customer in the clean_profile dataframe and create the relevant datapoints
    
    for index, row in profile_df.iterrows():
        #create 3 dataframes with received, viewed and completed offers of the customer
        
        #store customer id
        cust_id = row['id']
        #filter out customers transactions from the transcript
        cust_transactions = transcript_df.where(transcript_df['person'] == str(cust_id))
        cust_transactions.dropna(axis = 0, inplace = True)
        #split the customer transactions into 3 datasets for received, completed and viewed offers
        #received-df
        received_df = cust_transactions.where(cust_transactions['event'] == 'offer received')
        received_df.dropna(axis = 0, inplace = True)
        #viewed-df
        viewed_df = cust_transactions.where(cust_transactions['event'] == 'offer viewed')
        viewed_df.dropna(axis = 0, inplace = True)
        #completed-df
        completed_df = cust_transactions.where(cust_transactions['event'] == 'offer completed')
        completed_df.dropna(axis = 0, inplace = True)
        
        #call the create_customer_offers function to create the dataframe entries for the customer
        cust_entries = create_customer_offers(received_df, viewed_df, completed_df, portfolio)
        
        #append the customer entries to offer_df 
        offer_df = offer_df.append(cust_entries)
    
    
    return offer_df
    
    

In [221]:
#test function
import time
test_profile = clean_profile.head(100)
test_transcript = clean_transcript.copy()

%time result = create_offer_overview(test_profile, test_transcript, portfolio)

result.head(20)

CPU times: user 25.1 s, sys: 24 ms, total: 25.2 s
Wall time: 25.2 s


,person,offer_id,viewed,completed
0,0610b486422d4921ae7d2bf64640c50b,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,1
1,0610b486422d4921ae7d2bf64640c50b,3f207df678b143eea3cee63160fa8bed,0,0
0,78afa995795e4d85b5d9ceeca43f5fef,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1
1,78afa995795e4d85b5d9ceeca43f5fef,5a8bc65990b245e5a138643cd4eb9837,1,0
2,78afa995795e4d85b5d9ceeca43f5fef,ae264e3637204a6fb9bb56bc8210ddfd,1,1
3,78afa995795e4d85b5d9ceeca43f5fef,f19421c1d4aa40978ebb69ca19b0e20d,0,1
0,e2127556f4f64592b11af22de27a7932,2906b810c7d4411798c6938adc9daaa5,1,0
1,e2127556f4f64592b11af22de27a7932,3f207df678b143eea3cee63160fa8bed,0,0
2,e2127556f4f64592b11af22de27a7932,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1
3,e2127556f4f64592b11af22de27a7932,fafdcd668e3743c1bb461111dcafc2a4,0,1


In [222]:
# Create the new offer_df dataset with the create_offer_overview function
offer_df = create_offer_overview(clean_profile, clean_transcript, portfolio)

Checkpoint: Save the offer_df file to csv in order to have acces to it when restarting the notebook

In [226]:
#save offer_df file locally
offer_df.to_csv('data/Local Save Files/offer_df.csv')

In [238]:
#load offer_df file if necessary
offer_df_test = pd.read_csv('data/Local Save Files/offer_df.csv', index_col = 0)

The next step is to transform the viewed and completed column and add a label column that assigns the labels to the four categories of the viewed/completed matrix. The categories will be ordered from most to least desirable, i.e.:

0 = viewed & completed; 
1 = viewed & not completed; 
2 = not viewed & not completed; 
3 = not viewed & completed;

In [250]:
#define a function to map the values in the viewed & completed column to their labels
def mapping_func(row):
    #map values to categories
    if row['viewed'] == 1 and row['completed'] == 1:
        return 0
    if row['viewed'] == 1 and row['completed'] == 0:
        return 1
    if row['viewed'] == 0 and row['completed'] == 0:
        return 2
    if row['viewed'] == 0 and row['completed'] == 1:
        return 3
    #check if error
    return np.nan

#apply function to the offer_df dataframe
offer_df['label'] = offer_df.apply(lambda row: mapping_func(row), axis=1)

#Sanity Check
print('Missing values in labels column: {}'.format(offer_df.isnull().sum().sum()))
offer_df.head()

Missing values in labels column: 0


,person,offer_id,viewed,completed,label
0,0610b486422d4921ae7d2bf64640c50b,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,1,3
1,0610b486422d4921ae7d2bf64640c50b,3f207df678b143eea3cee63160fa8bed,0,0,2
0,78afa995795e4d85b5d9ceeca43f5fef,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,0
1,78afa995795e4d85b5d9ceeca43f5fef,5a8bc65990b245e5a138643cd4eb9837,1,0,1
2,78afa995795e4d85b5d9ceeca43f5fef,ae264e3637204a6fb9bb56bc8210ddfd,1,1,0


Now that the order_df dataframe is complete, the next step is to extend the dataframe to include the information about the offer. This can be done similarly to the mapping of the labels by taking the information from the portfolio dataframe and the offer_id in the offer_df dataframe. As a result a new dataframe dataframe __offer_df_extended__ will be created. __The features are not yet scaled!!__

In [257]:
# define function to add information from the portfolio dataframe based on the offer_id
def add_offer_info(row, portfolio, column):
    return portfolio.loc[portfolio['id'] == row['offer_id']][column].squeeze()

# add the relevant columns to the offer_df dataset in a new dataframe offer_df_extended
offer_df_extended = offer_df
# add channels
offer_df_extended['offer_channels'] = offer_df_extended.apply(lambda row: add_offer_info(row, portfolio, 'channels'), axis=1)
# add type
offer_df_extended['offer_type'] = offer_df_extended.apply(lambda row: add_offer_info(row, portfolio, 'offer_type'), axis=1)
# add reward
offer_df_extended['offer_reward'] = offer_df_extended.apply(lambda row: add_offer_info(row, portfolio, 'reward'), axis=1)
#add difficulty
offer_df_extended['offer_difficulty'] = offer_df_extended.apply(lambda row: add_offer_info(row, portfolio, 'difficulty'), axis=1)
#add duration of offer
offer_df_extended['offer_duration'] = offer_df_extended.apply(lambda row: add_offer_info(row, portfolio, 'duration'), axis=1)

In [258]:
offer_df_extended.head(20)

,person,offer_id,viewed,completed,label,offer_channels,offer_type,offer_reward,offer_difficulty,offer_duration
0,0610b486422d4921ae7d2bf64640c50b,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,1,3,"[web, email, mobile]",bogo,5,5,7
1,0610b486422d4921ae7d2bf64640c50b,3f207df678b143eea3cee63160fa8bed,0,0,2,"[web, email, mobile]",informational,0,0,4
0,78afa995795e4d85b5d9ceeca43f5fef,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,0,"[web, email, mobile]",bogo,5,5,7
1,78afa995795e4d85b5d9ceeca43f5fef,5a8bc65990b245e5a138643cd4eb9837,1,0,1,"[email, mobile, social]",informational,0,0,3
2,78afa995795e4d85b5d9ceeca43f5fef,ae264e3637204a6fb9bb56bc8210ddfd,1,1,0,"[email, mobile, social]",bogo,10,10,7
3,78afa995795e4d85b5d9ceeca43f5fef,f19421c1d4aa40978ebb69ca19b0e20d,0,1,3,"[web, email, mobile, social]",bogo,5,5,5
0,e2127556f4f64592b11af22de27a7932,2906b810c7d4411798c6938adc9daaa5,1,0,1,"[web, email, mobile]",discount,2,10,7
1,e2127556f4f64592b11af22de27a7932,3f207df678b143eea3cee63160fa8bed,0,0,2,"[web, email, mobile]",informational,0,0,4
2,e2127556f4f64592b11af22de27a7932,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,0,"[web, email, mobile]",bogo,5,5,7
3,e2127556f4f64592b11af22de27a7932,fafdcd668e3743c1bb461111dcafc2a4,0,1,3,"[web, email, mobile, social]",discount,2,10,10


Now that we have all the information about the offers in one dataframe, the customer information will be constructed and added. For this the transaction volume per customer per week will be added to the profile dataframe. This information will then later be added to the offer_df_extended dataframe.